Attempting to solve the Multi-Armed Bandit problem

In [1]:
import tensorflow as tf
import numpy as np

<h3>The Bandits</h3>
Here we define our bandits. For this example we are using a four-armed bandit. The pullBandit function generates a random number from a normal distribution with a mean of 0. The lower the bandit number, the more likely a positive reward will be returned. We want our agent to learn to always choose the bandit that will give that positive reward.

In [24]:
b1 = round(5 * np.random.randn(), 2)
b2 = round(5 * np.random.randn(), 2)
b3 = round(5 * np.random.randn(), 2)
b4 = round(5 * np.random.randn(), 2)

bandits = [b1, b2, b3, b4]
num_bandits = len(bandits)

def pullBandit(bandit):
    result = np.random.randn(1)
    if result > bandit:
        return 1
    else:
        return -1

<h3>The Agent</h3>
The code below established our simple neural agent. It consists of a set of values for each of the bandits. Each value is an estimate of the value of the return from choosing the bandit. We use a policy gradient method to update the agent by moving the value for the selected action toward the recieved reward.

In [25]:
tf.reset_default_graph()

weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights, 0)

reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)
action_holder = tf.placeholder(shape=[1], dtype=tf.int32)
responsible_weight = tf.slice(weights, action_holder, [1])
loss = -(tf.log(responsible_weight) * reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

<h3>Training the Agent</h3>
We will train our agent by taking actions in our environment, and recieving rewards. Using the rewards and actions, we can know how to properly update our network in order to more often choose actions that will yield the highest rewards over time.

In [27]:
print("Bandits:", bandits)

total_episodes = 1000
total_reward = np.zeros(num_bandits) #'scoreboard' for bandits
e = 0.1 #chance of taking random action

init = tf.global_variables_initializer()

with tf.Session() as sess:

    sess.run(init)

    for i in range(total_episodes):
        
        if np.random.rand(1) < e: #for random action
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
        
        # Get reward from chosen Bandit
        reward = pullBandit(bandits[action])
        
        _, resp, ww = sess.run([update, responsible_weight, weights],
                              feed_dict={reward_holder:[reward], action_holder:[action]})
        
        total_reward[action] += reward
        
        if i % 50 == 0:
            print ("Running reward for the " + str(num_bandits) + " bandits: " + str(total_reward))
        

print ("The agent thinks bandit " + str(np.argmax(ww)+1) + " is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print ("...and it was right!")
else:
    print ("...and it was wrong!")

Bandits: [2.49, -3.51, 7.6, 1.35]
Running reward for the 4 bandits: [-1.  0.  0.  0.]
Running reward for the 4 bandits: [-4. 43. -2. -2.]
Running reward for the 4 bandits: [-4. 90. -2. -5.]
Running reward for the 4 bandits: [ -6. 136.  -2.  -7.]
Running reward for the 4 bandits: [-10. 180.  -3.  -8.]
Running reward for the 4 bandits: [-11. 227.  -4.  -9.]
Running reward for the 4 bandits: [-15. 271.  -4.  -9.]
Running reward for the 4 bandits: [-15. 318.  -6. -10.]
Running reward for the 4 bandits: [-15. 363.  -8. -13.]
Running reward for the 4 bandits: [-16. 408. -10. -15.]
Running reward for the 4 bandits: [-16. 454. -12. -17.]
Running reward for the 4 bandits: [-19. 496. -17. -17.]
Running reward for the 4 bandits: [-20. 541. -17. -21.]
Running reward for the 4 bandits: [-22. 586. -19. -22.]
Running reward for the 4 bandits: [-22. 634. -19. -24.]
Running reward for the 4 bandits: [-23. 682. -20. -24.]
Running reward for the 4 bandits: [-24. 730. -21. -24.]
Running reward for the 4 b